# Building a simple Agent with Tools and Toolkits in LangChain

## Configuring the environment and installing  

First create a new Environment using the menu. Add the following information in the new environment:  

``` 
Environment Variables Set Name : OpenAI  
NAME    : OPENAI_API_KEY 
VALUE  : ```

## What are Agents

If you lookup the definition of AI Agents, you get something along the lines of “An entity that is able to perceive its environment, act on its environment, and make intelligent decisions about how to reach a goal it has been given, as well as the ability to learn as it goes”

That fits the definition of LangChain agents pretty well. What makes all this possible in software is the reasoning abilities of Large Language Model’s (LLM’s). The brains of a LangChain agent are an LLM. It is the LLM that is used to reason about the best way to carry out the ask requested by a user.

In order to carry out its task, and operate on things and retrieve information, the agent has `Tool`s. It is through these tools that it is able to interact with its environment.

The tools are basically just methods/classes the agent has access to that can do things like interact with a Stock Market index over an API, update a Google Calendar event, or run a query against a database. We can build out tools as needed, depending on the nature of tasks we are trying to carry out with the agent to fulfil.

A collection of Tools in LangChain are called a Toolkit. Implementation wise, this is literally just an array of the Tools that are available for the agent.

So, at a basic level, an agent needs

* an LLM to act as its brain, and to give it its reasoning abilities
* tools so that it can interact with the environment around it and achieve its goals


## Building the Agent
To make some of these concepts more concrete, let’s build a simple agent.

We will create a Mathematics Agent that can perform a few simple mathematical operations.

First let's install the required packages:

In [1]:
pip install langchain langchain_openai

  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached jiter-0.8.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 581.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl (982 kB)
Using cached jiter-0.8.2-cp312-cp312-macosx_11_0_arm64.whl (310 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.19.0
    Uninstalling openai-1.19.0:
      Successfully uninstalled openai-1.19.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
ERROR: pip's dependency resolver does not currently take into ac

> Please restart the kernal after the installations: `Run -> Restart Kernal`

Just execute the cell below to import the required packages:

In [ ]:
OPENAI_API_KEY = ""

In [1]:
import os

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

/Users/dogukanince/miniforge3/envs/test_env/lib/python3.12/site-packages/langchain/chains/api/base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper
/Users/dogukanince/miniforge3/envs/test_env/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `TextRequestsWrapper` to V2.
  warn(


## The Tools
The simplest place to start will be to fist define the tools for our Maths agent.

Let’s give it “add”, “multiply” and “square” tools, so that it can perform those operations on questions we pass to it. By keeping our tools simple we can focus on the core concepts, and build the tools ourselves, instead of relying on an existing and more complex tools like the WikipediaTool, that acts as a wrapper around the Wikipedia API, and requires us to import it from the LangChain library.

Again, we are not trying to do anything fancy here, just keeping it simple and putting the main building blocks of an agent together so we can understand how they work, and get our first agent up and running.

Let’s start with the “add” tool. LangChain does give us an easier way to define tools, then by needing to extend the BaseTool class each time. We can do this with the help of the @tool decorator. Defining the “add” tool in LangChain using the @tool decorator will look like this

In [7]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


Some thing to note:

* the method name also becomes the tool name
* the method params define the input parameters for the tool
* the docstring gets converted into the tools description

You can access these properties (`name`, `description`, and `args`) on the tool also:

In [8]:
print(add.name) 
print(add.description)
print(add.args) 

add
Add two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


Note that the description of a tool is very important as this is what the LLM uses to decide whether or not it is the right tool for the job. A bad description may lead to the not tool getting used when it should be, or getting used at the wrong times.

With the add tool done, let’s move on to the definitions for our multiply and square tools.

Please below define both `multiply` and `square` functiosn as tools:

In [9]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def square(a: int) -> int:
    """Square a number."""
    return a ** 2

And that is it, simple as that.

So we have defined our own three [custom tools](https://python.langchain.com/docs/modules/tools/custom_tools/). A more common use case might be to use some of the already provided and existing tools in LangChain, which you can see [here](https://python.langchain.com/docs/integrations/tools/). However, at the source code level, they would all be built and defined using a similar methods as described above.

And that is it as far as our Tools our concerned. Now time to combine our tools into a Toolkit.

## The Toolkit
Toolkits sound fancy, but they are actually very simple. They are literally just a a list of tools. We can define our toolkit as a list of tools.

In the list below, fill in the blanks with the name of the tools you defined above.

In [10]:
toolkit = [add, multiply, square]

And that’s it. Really straightforward, and nothing to get confused over.

Usually Toolkits are groups of tools that are useful together, and would be helpful for agents trying to carry out certain kinds of tasks. For example an SQLToolkit might contain a tool for generating an SQL query, validating an SQL query, and executing an SQL query.

The [Integrations Toolkit](https://python.langchain.com/docs/integrations/toolkits/) page on the LangChain docs has a large list of toolkits developed by the community that might be useful for you.

## The LLM

As mentioned above, an LLM is the brains of an agent. It decides which tools to call based on the question passed to it, what are the best next steps to take based on a tools description. It also decides when it has reached its final answer, and is ready to return that to the user.

Let’s setup the LLM here. The ChatOpenAI receives two parameters `model` and `temperature`. As the model you can use `"gpt-3.5-turbo-1106"`. Temperature typically ranges from -1 to 1, you can choose any temperature value.

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model = "gpt-3.5-turbo-1106", 
    temperature=0.9)

## The Prompt
Lastly we need a prompt to pass into our agent, so it has a general idea about what kind of agent it is, and what sorts of tasks it should solve.

Our agent requires a ChatPromptTemplate to work (more on that later). This is what a barebones ChatPromptTemplate looks like. The main part we care about is the system prompt, and the rest are just the default settings we are required to pass in.

In our prompt, we have included a sample answer, showing the agent how we want it to return the answer only, and not any descriptive text along with the answer:

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a mathematical assistant. Use your tools to answer questions.
         If you do not have a tool to answer the question, say so.

        Return only the answers. e.g
        Human: What is 1 + 1?
        AI: 2
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


That is it. We have setup our Tools and Toolkit, which our agent will need as part of its setup, so its knows what are the types of actions and capabilities it has at its disposal. And we have also setup the LLM and system prompt.

Now for the fun part. Setting up our Agent!

## The Agent

LangChain has a number of different agents types that can be created, with different reasoning powers and abilities. We will be using the most capable and powerful agent currently available, the OpenAI Tools agent. As per the docs on the the OpenAI Tools agent, which uses newer OpenAI models also,

> Newer OpenAI models have been fine-tuned to detect when one or more function(s) should be called and respond with the inputs that should be passed to the function(s). In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call these functions. The goal of the OpenAI tools APIs is to more reliably return valid and useful function calls than what can be done using a generic text completion or chat API.

In other words this agents is good at generating the correct structure for calling functions, and is able to understand if more than one function (tool) might be needed for our task also. This agent also has the ability to call functions (tools) with multiple input parameters, just like ours do. Some agents can only work with functions that have a single input parameter.

If you are familiar with OpenAI’s Function calling feature, where we can use the OpenAI LLM to generate the correct parameters to call a function with, the OpenAI Tools agent we are using here is leveraging some of that power in order to be able to call the correct tool, with the correct parameters.

In order to setup an agent in LangChain, we need to use one of the factory methods provided for creating the agent of our choice.

The factory method for creating an OpenAI tools agent is `create_openai_tools_agent()`. And it requires passing in the llm, tools and prompt we setup above. So let’s initialise our agent.

In [15]:
agent = create_openai_tools_agent(llm, toolkit, prompt)

Finally, in order to run agents in LangChain, we cannot just call a “run” type method on them directly. They need to be run via an AgentExecutor.

I am bringing up the Agent Executor only here at the end as I don’t think it’s a critical concept for understanding how the agents work, and bring it up at the start with everything else would just the whole thing seem more complicated than it needs to be, as well as distract from understanding some of the other more fundamental concepts.

So, now that we are introducing it, an `AgentExecutor` acts as the runtime for agents in LangChain, and allow an agent to keep running until it is ready to return its final response to the user. In pseudo-code, the AgentExecutor’s are doing something along the lines of (pulled directly from the LangChain docs):
```
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action
```

So they are basically a while loop that keep’s calling the next action methods on the agent, until the agent has returned its final response.

So, let us setup our agent inside the agent executor. We pass it the agent, and must also pass it the toolkit. And we are setting verbose to True so we can get an idea of what the agent is doing as it is processing our request

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

And that is it. We are now ready to pass commands to our agent

In [17]:
result = agent_executor.invoke({"input": "what is 1 + 3?"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 3}`


4The sum of 1 and 3 is 4.

> Finished chain.
The sum of 1 and 3 is 4.


Since we have set verbose=True on the AgentExecutor, we can see the lines of Action our agent has taken. It has identified we should call the “add” tool, called the “add” tool with the required parameters, and returned us our result.

Please make several further trials to ensure that the LLM chooses the correct tool each time:

In [18]:
result = agent_executor.invoke({"input": "what is 2 * 3?"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 3}`


62 multiplied by 3 is equal to 6.

> Finished chain.
2 multiplied by 3 is equal to 6.
